In [1]:
# --- Project Initialization and Environment Setup for SQL Analysis ---
import sys
import os
from pathlib import Path

# --- Add Project Root to sys.path ---
project_root = Path.cwd().parent.resolve()
project_root_str = str(project_root)
if project_root_str not in sys.path:
    sys.path.insert(0, project_root_str)

# --- Core Imports ---
import logging
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import text

# Project-specific import
from src.database.db_connection import get_database_engine

# --- Configuration and Logging ---
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("sql_analysis_notebook")

OUTPUT_DIR = Path("output") # Relative to notebooks/
OUTPUT_DIR.mkdir(exist_ok=True)

print("✅ Environment initialized for SQL analysis.")

# --- Database Connection ---
try:
    engine = get_database_engine()
    logger.info("Database engine acquired for SQL analysis.")
    with engine.connect() as conn:
        result = conn.execute(text("SELECT current_database(), current_user;"))
        db_info_row = result.fetchone()
        if db_info_row:
            db_name, db_user = db_info_row
            print(f"✅ Connected to database '{db_name}' as user '{db_user}'.")
        else:
            print("⚠️  Connected to database (info retrieval returned no rows).")
except Exception as e:
    logger.error(f"Database connection failed: {e}", exc_info=True)
    print(f"❌ Error connecting to database: {e}")
    raise


INFO:src.database.db_connection:Creating database engine for database=banking_analytics_db host=localhost port=5432 driver=postgresql+psycopg
INFO:sql_analysis_notebook:Database engine acquired for SQL analysis.


✅ Environment initialized for SQL analysis.
✅ Connected to database 'banking_analytics_db' as user 'bank_user'.


# Advanced SQL Data Profiling & Analysis

This notebook leverages advanced SQL queries to perform detailed profiling and analysis directly on the PostgreSQL database. This approach is efficient for large datasets and complements Python-based analysis.

**Analysis Focus:**
*   Detailed numerical profiling (percentiles, distribution)
*   SQL-based outlier detection
*   Categorical data distribution
*   Time-series trends
*   Basic correlations and data validation

## 1. Detailed Numerical Profiling

Calculate detailed statistics for key numerical columns like `transactions.amount` and `accounts.balance`.

In [2]:
# --- 1. Detailed Numerical Profiling ---

# --- Profiling: transactions.amount ---
print("--- Profiling: transactions.amount ---")
sql_amount_profile = """
SELECT
    COUNT(amount) AS count,
    AVG(amount) AS mean,
    STDDEV(amount) AS stddev,
    MIN(amount) AS min,
    MAX(amount) AS max,
    -- Calculate percentiles (approximate for large datasets)
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY amount) AS q1,
    PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY amount) AS median,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY amount) AS q3
FROM transactions;
"""
try:
    df_amount_profile = pd.read_sql(text(sql_amount_profile), engine)
    print("Statistical Profile for 'transactions.amount':")
    display(df_amount_profile)
except Exception as e:
    logger.error(f"Error executing amount profile query: {e}")
    print(f"❌ Error: {e}")

print("\n" + "-"*20 + "\n")

# --- Profiling: accounts.balance ---
print("--- Profiling: accounts.balance ---")
sql_balance_profile = """
SELECT
    COUNT(balance) AS count,
    AVG(balance) AS mean,
    STDDEV(balance) AS stddev,
    MIN(balance) AS min,
    MAX(balance) AS max,
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY balance) AS q1,
    PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY balance) AS median,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY balance) AS q3
FROM accounts;
"""
try:
    df_balance_profile = pd.read_sql(text(sql_balance_profile), engine)
    print("Statistical Profile for 'accounts.balance':")
    display(df_balance_profile)
except Exception as e:
    logger.error(f"Error executing balance profile query: {e}")
    print(f"❌ Error: {e}")


--- Profiling: transactions.amount ---
Statistical Profile for 'transactions.amount':


,count,mean,stddev,min,max,q1,median,q3
0,4912,564.11384,978.279227,3.99,16211.03,108.1125,271.235,645.22



--------------------

--- Profiling: accounts.balance ---
Statistical Profile for 'accounts.balance':


,count,mean,stddev,min,max,q1,median,q3
0,627,7118.250925,8618.168187,-7517.83,63966.78,1914.065,4801.43,9344.73


## 2. SQL-Based Outlier Detection

Identify potential outliers in `transactions.amount` using the Interquartile Range (IQR) method calculated within SQL.

In [3]:
# --- 2. SQL-Based Outlier Detection (IQR Method) ---
print("--- SQL-Based Outlier Detection (IQR) for transactions.amount ---")

# Calculate IQR bounds and identify outliers in a single query
sql_outliers = """
WITH quartiles AS (
    SELECT
        PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY amount) AS q1,
        PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY amount) AS q3
    FROM transactions
),
iqr_bounds AS (
    SELECT
        q1,
        q3,
        (q3 - q1) AS iqr,
        (q1 - 1.5 * (q3 - q1)) AS lower_bound,
        (q3 + 1.5 * (q3 - q1)) AS upper_bound
    FROM quartiles
)
SELECT
    t.transaction_id,
    t.account_id,
    t.amount,
    t.transaction_type,
    i.lower_bound,
    i.upper_bound
FROM transactions t
CROSS JOIN iqr_bounds i
WHERE t.amount < i.lower_bound OR t.amount > i.upper_bound
ORDER BY t.amount DESC
LIMIT 20; -- Show top 20 potential outliers
"""
try:
    df_sql_outliers = pd.read_sql(text(sql_outliers), engine)
    print(f"Found {len(df_sql_outliers)} potential outliers (showing top 20):")
    display(df_sql_outliers)
    print(f"\nTotal Outliers (based on IQR): {len(df_sql_outliers)}")
except Exception as e:
    logger.error(f"Error executing SQL outlier detection query: {e}")
    print(f"❌ Error: {e}")


--- SQL-Based Outlier Detection (IQR) for transactions.amount ---
Found 20 potential outliers (showing top 20):


,transaction_id,account_id,amount,transaction_type,lower_bound,upper_bound
0,TXN_0000070076,ACC_0000587,16211.03,Deposit,-697.54875,1450.88125
1,TXN_0000072401,ACC_0000606,15859.44,Deposit,-697.54875,1450.88125
2,TXN_0000074830,ACC_0000627,14855.27,Deposit,-697.54875,1450.88125
3,TXN_0000073731,ACC_0000618,13714.75,Deposit,-697.54875,1450.88125
4,TXN_0000072464,ACC_0000607,13239.46,Deposit,-697.54875,1450.88125
5,TXN_0000074041,ACC_0000620,12723.04,Withdrawal,-697.54875,1450.88125
6,TXN_0000071839,ACC_0000602,11018.30,Deposit,-697.54875,1450.88125
7,TXN_0000070449,ACC_0000590,10742.02,Deposit,-697.54875,1450.88125
8,TXN_0000071980,ACC_0000603,8812.34,Withdrawal,-697.54875,1450.88125
9,TXN_0000073096,ACC_0000612,8357.93,Transfer_In,-697.54875,1450.88125



Total Outliers (based on IQR): 20


## 3. Categorical Data Distribution

Analyze the distribution of categorical variables like `transaction_type`, `account_type`, and `region`.

In [4]:
# --- 3. Categorical Data Distribution ---
print("--- Categorical Data Distribution ---")

# --- Distribution: transaction_type ---
print("\n--- Distribution: transaction_type ---")
sql_txn_type_dist = """
SELECT transaction_type, COUNT(*) AS count
FROM transactions
GROUP BY transaction_type
ORDER BY count DESC;
"""
try:
    df_txn_type_dist = pd.read_sql(text(sql_txn_type_dist), engine)
    print("Transaction Type Distribution:")
    display(df_txn_type_dist)
except Exception as e:
    logger.error(f"Error executing transaction type distribution query: {e}")
    print(f"❌ Error: {e}")

# --- Distribution: account_type ---
print("\n--- Distribution: account_type ---")
sql_acc_type_dist = """
SELECT account_type, COUNT(*) AS count
FROM accounts
GROUP BY account_type
ORDER BY count DESC;
"""
try:
    df_acc_type_dist = pd.read_sql(text(sql_acc_type_dist), engine)
    print("Account Type Distribution:")
    display(df_acc_type_dist)
except Exception as e:
    logger.error(f"Error executing account type distribution query: {e}")
    print(f"❌ Error: {e}")

# --- Distribution: region ---
print("\n--- Distribution: region (from customers) ---")
sql_region_dist = """
SELECT region, COUNT(*) AS customer_count
FROM customers
GROUP BY region
ORDER BY customer_count DESC;
"""
try:
    df_region_dist = pd.read_sql(text(sql_region_dist), engine)
    print("Customer Region Distribution:")
    display(df_region_dist)
except Exception as e:
    logger.error(f"Error executing region distribution query: {e}")
    print(f"❌ Error: {e}")


--- Categorical Data Distribution ---

--- Distribution: transaction_type ---
Transaction Type Distribution:


,transaction_type,count
0,Payment,1333
1,Deposit,1315
2,Withdrawal,1137
3,Transfer_In,566
4,Transfer_Out,561



--- Distribution: account_type ---
Account Type Distribution:


,account_type,count
0,Savings,321
1,Checking,247
2,Credit,59



--- Distribution: region (from customers) ---
Customer Region Distribution:


,region,customer_count
0,Qatar_East,116
1,Doha_Central,103
2,Qatar_West,94
3,Qatar_South,94
4,Qatar_North,93


## 4. Time-Series Analysis

Examine transaction trends over time (e.g., monthly counts, average amounts).

In [5]:
# --- 4. Time-Series Analysis ---
print("--- Time-Series Analysis ---")

# --- Transactions per Month ---
print("\n--- Transactions per Month ---")
sql_txn_per_month = """
SELECT
    DATE_TRUNC('month', transaction_date) AS month,
    COUNT(*) AS transaction_count,
    AVG(amount) AS avg_amount
FROM transactions
GROUP BY DATE_TRUNC('month', transaction_date)
ORDER BY month;
"""
try:
    df_txn_per_month = pd.read_sql(text(sql_txn_per_month), engine)
    # Convert month to datetime for easier handling if needed
    df_txn_per_month['month'] = pd.to_datetime(df_txn_per_month['month'])
    print("Monthly Transaction Counts and Average Amounts:")
    display(df_txn_per_month.head(10)) # Show first 10 months

    # Optional: Plot within notebook (basic example)
    # import matplotlib.pyplot as plt
    # fig, ax1 = plt.subplots(figsize=(10, 5))
    # ax1.set_xlabel('Month')
    # ax1.set_ylabel('Transaction Count', color='tab:blue')
    # ax1.plot(df_txn_per_month['month'], df_txn_per_month['transaction_count'], color='tab:blue', marker='o')
    # ax1.tick_params(axis='y', labelcolor='tab:blue')
    # ax2 = ax1.twinx()
    # ax2.set_ylabel('Average Amount', color='tab:red')
    # ax2.plot(df_txn_per_month['month'], df_txn_per_month['avg_amount'], color='tab:red', marker='x')
    # ax2.tick_params(axis='y', labelcolor='tab:red')
    # fig.tight_layout()
    # plt.title('Transactions per Month & Average Amount')
    # plt.show()

except Exception as e:
    logger.error(f"Error executing time-series analysis query: {e}")
    print(f"❌ Error: {e}")


--- Time-Series Analysis ---

--- Transactions per Month ---
Monthly Transaction Counts and Average Amounts:


,month,transaction_count,avg_amount
0,2024-08-01,167,565.069102
1,2024-09-01,426,585.476502
2,2024-10-01,446,588.074552
3,2024-11-01,412,570.434102
4,2024-12-01,447,600.067763
5,2025-01-01,421,539.664703
6,2025-02-01,405,509.033827
7,2025-03-01,398,663.854899
8,2025-04-01,391,539.372711
9,2025-05-01,403,473.479380


## 5. Correlation Analysis & Data Validation

Explore potential relationships and perform basic data validation checks using SQL.

In [6]:
# --- 5. Correlation Analysis & Data Validation ---
print("--- Correlation Analysis & Data Validation ---")

# --- Average Balance by Account Type ---
print("\n--- Average Balance by Account Type ---")
sql_avg_bal_by_type = """
SELECT account_type, AVG(balance) AS avg_balance, COUNT(*) AS account_count
FROM accounts
GROUP BY account_type
ORDER BY avg_balance DESC;
"""
try:
    df_avg_bal_by_type = pd.read_sql(text(sql_avg_bal_by_type), engine)
    print("Average Account Balance by Account Type:")
    display(df_avg_bal_by_type)
except Exception as e:
    logger.error(f"Error executing avg balance by type query: {e}")
    print(f"❌ Error: {e}")

# --- Data Validation: Check for obviously invalid data ---
print("\n--- Data Validation Checks ---")

# Check for negative amounts (if business rules allow only positive)
print("\n--- Check for Negative Transaction Amounts ---")
sql_negative_amounts = """
SELECT COUNT(*) AS negative_amount_count
FROM transactions
WHERE amount < 0;
"""
try:
    df_negative_check = pd.read_sql(text(sql_negative_amounts), engine)
    neg_count = df_negative_check.iloc[0]['negative_amount_count']
    if neg_count > 0:
        print(f"⚠️  WARNING: Found {neg_count} transactions with negative amounts.")
    else:
        print("✅ No negative transaction amounts found.")
except Exception as e:
    logger.error(f"Error executing negative amount check query: {e}")
    print(f"❌ Error: {e}")

# Check for transactions in the future (data integrity)
print("\n--- Check for Future-Dated Transactions ---")
sql_future_txns = """
SELECT COUNT(*) AS future_txn_count
FROM transactions
WHERE transaction_date > NOW();
"""
try:
    df_future_check = pd.read_sql(text(sql_future_txns), engine)
    future_count = df_future_check.iloc[0]['future_txn_count']
    if future_count > 0:
        print(f"⚠️  WARNING: Found {future_count} transactions dated in the future.")
    else:
        print("✅ No future-dated transactions found.")
except Exception as e:
    logger.error(f"Error executing future date check query: {e}")
    print(f"❌ Error: {e}")

# --- Optional: Join Analysis (e.g., Transactions linked to Regions) ---
print("\n--- Transaction Count by Customer Region (via Join) ---")
sql_txn_by_region = """
SELECT c.region, COUNT(t.transaction_id) AS transaction_count
FROM transactions t
JOIN accounts a ON t.account_id = a.account_id
JOIN customers c ON a.customer_id = c.customer_id
GROUP BY c.region
ORDER BY transaction_count DESC;
"""
try:
    df_txn_by_region = pd.read_sql(text(sql_txn_by_region), engine)
    print("Transaction Count by Customer Region:")
    display(df_txn_by_region)
except Exception as e:
    logger.error(f"Error executing transaction by region query: {e}")
    print(f"❌ Error: {e}")


--- Correlation Analysis & Data Validation ---

--- Average Balance by Account Type ---
Average Account Balance by Account Type:


,account_type,avg_balance,account_count
0,Savings,10579.750405,321
1,Checking,4822.760121,247
2,Credit,-2104.716949,59



--- Data Validation Checks ---

--- Check for Negative Transaction Amounts ---
✅ No negative transaction amounts found.

--- Check for Future-Dated Transactions ---
✅ No future-dated transactions found.

--- Transaction Count by Customer Region (via Join) ---
Transaction Count by Customer Region:


,region,transaction_count
0,Doha_Central,1306
1,Qatar_East,1201
2,Qatar_North,847
3,Qatar_South,841
4,Qatar_West,717
